# Pairwise Adjusted Mutual Information

# Experiments on real data

This notebook presents the experiments on real data.

## Imports

In [1]:
import warnings
import glob
from natsort import natsorted
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import mutual_info_score
from sklearn.metrics.cluster._expected_mutual_info_fast import expected_mutual_information
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD

In [ ]:
warnings.filterwarnings("ignore")

## Data

We consider the following benchmark, consisting of **79** different datasets:

M. Gagolewski and others (Eds.), Benchmark Suite for Clustering Algorithms -- Version 1, 2020

https://github.com/gagolews/clustering_benchmarks_v1

In [ ]:
!git clone https://github.com/gagolews/clustering_benchmarks_v1

In [ ]:
path = "./clustering_benchmarks_v1/"
data_files = natsorted([f for f in glob.glob(f"{path}*/*.data.gz")])
dataset_names = [file.split(path)[1].split('.')[0] for file in data_files]
dataset_names = [name for name in dataset_names if not (('g2mg' in name) or ('h2mg' in name))]

In [ ]:
len(dataset_names)

## Pairwise Adjusted Mutual Information

In [ ]:
def get_adjusted_mutual_info_pair(contingency, n_samples):
    """Return pairwise adjusted mutual information.
    
    Parameters
    ----------
    contingency: np.ndarray
        Contingency matrix
    n_samples : int
        Number of samples
    """
    k, l = contingency.shape
    a = contingency.sum(axis=1)
    b = contingency.sum(axis=0)
    c = contingency.ravel()
    # first term
    factor = c * (contingency - np.outer(a, np.ones(l)) - np.outer(np.ones(k), b) + n_samples).ravel()
    entropy = np.zeros(len(c))
    entropy[c > 0] = c[c > 0] / n_samples * np.log(c[c > 0] / n_samples)
    entropy_ = np.zeros(len(c))
    entropy_[c > 1] = (c[c > 1] - 1) / n_samples * np.log((c[c > 1] - 1) / n_samples)
    result = np.sum(factor * (entropy - entropy_)) / n_samples ** 2
    # second term
    factor = ((np.outer(a, np.ones(l)) - contingency) * (np.outer(np.ones(k), b) - contingency)).ravel()
    entropy_ = (c + 1) / n_samples * np.log((c + 1) / n_samples)
    result += np.sum(factor * (entropy - entropy_)) / n_samples ** 2
    return result

## Full Adjusted Mutual Information

In [ ]:
def get_adjusted_mutual_info_exact(contingency, n_samples):
    """Return adjusted mutual information (without normalization).
    
    Parameters
    ----------
    contingency: np.ndarray
        Contingency matrix
    n_samples : int
        Number of samples
    """
    mi = mutual_info_score(_, _, contingency=contingency)
    emi = expected_mutual_information(contingency, n_samples)
    result = mi - emi
    return result

## Clustering

We consider the clustering algorithms of scikit-learn.

In order to evaluate similarity between results obtained with **Full Adjusted Mutual Information** and **Pairwise Adjusted Mutual Information**, we use the **Spearman correlation** between rankings.

In [ ]:
default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

In [ ]:
def prepare_algorithms(X):
    params = default_base.copy()
    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])
    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=params['eps'])
    optics = cluster.OPTICS(min_samples=params['min_samples'],
                            xi=params['xi'],
                            min_cluster_size=params['min_cluster_size'])
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')
    return (
                ('MiniBatchKMeans', two_means),
                ('AffinityPropagation', affinity_propagation),
                ('MeanShift', ms),
                ('SpectralClustering', spectral),
                ('Ward', ward),
                ('AgglomerativeClustering', average_linkage),
                ('DBSCAN', dbscan),
                ('OPTICS', optics),
                ('Birch', birch),
                ('GaussianMixture', gmm)
            )

## Test

In [ ]:
# set the seed for reproducible results
np.random.seed(0)

In [ ]:
nb_samples = []
nb_labels = []
results = []
gains = []

In [ ]:
for i, name in enumerate(dataset_names):
    
    X = np.loadtxt(path + name +".data.gz", ndmin=2)
    y = np.loadtxt(path + name +".labels0.gz", dtype=np.intc)
    n_samples, n_features = X.shape
    nb_samples.append(n_samples)
    nb_labels.append(len(set(y)))
    
    print(i + 1, '/', len(dataset_names), name, n_samples)
    
    if n_features > 100:
        # dimension reduction
        svd = TruncatedSVD(n_components=10)
        X = svd.fit_transform(X)

    X = StandardScaler().fit_transform(X)
    clustering_algorithms = prepare_algorithms(X)

    sim_full = []
    sim_pair = []
    time_full = []
    time_pair = []

    for algo_name, algorithm in clustering_algorithms:
        algorithm.fit(X)
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(int)
        else:
            y_pred = algorithm.predict(X)
        contingency = contingency_matrix(y, y_pred)
        t0 = time.time()
        sim_full.append(get_adjusted_mutual_info_exact(contingency, len(y)))
        t1 = time.time()
        time_full.append(t1 - t0)
        t0 = time.time()
        sim_pair.append(get_adjusted_mutual_info_pair(contingency, len(y)))
        t1 = time.time()        
        time_pair.append(t1 - t0)

    result = stats.spearmanr(sim_full, sim_pair).correlation
    gain = np.mean(np.array(time_full) / np.array(time_pair))

    results.append(result)
    gains.append(gain)

In [ ]:
# nb of datasets with correlation > 0.95
np.sum(np.array(results) > 0.95)

In [ ]:
index = np.argsort(nb_samples)

In [ ]:
plt.plot(1 + np.arange(len(results)), np.array(results)[index], c='b', lw=3)
plt.ylim(0,1)
plt.xlabel('Dataset')
plt.ylabel('Spearman correlation')
plt.savefig('spearman.pdf', bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
plt.plot(1 + np.arange(len(gains)), np.array(gains)[index], c='b', lw=3)
plt.ylim(0, 20)
plt.yticks([0,5, 10,15, 20])
plt.xlabel('Dataset')
plt.ylabel('Speed-up')
plt.savefig('speedup.pdf', bbox_inches='tight', transparent=True)
plt.show()